# Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer

/home/tim/.pyenv/versions/llm-zoomcamp/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [31]:
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

In [32]:
user_question = "I just discovered the course. Can I still join it?"

In [33]:
v_q = embedding_model.encode(user_question)

In [34]:
v_q[0]

0.078222655

The first value of the resulting vector is 0.07.

# Q2. Creating the embeddings

In [6]:
import requests
import numpy as np

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [7]:
ml_docs = [d for d in documents if d['course'] == "machine-learning-zoomcamp"]

In [8]:
len(ml_docs)

375

In [9]:
embeddings = []

In [10]:
for doc in ml_docs:
    question = doc['question']
    text = doc['text']
    qa_text = f'{question} {text}'
    embeddings.append(embedding_model.encode(qa_text))

In [11]:
X = np.array(embeddings)

In [12]:
X.shape

(375, 768)

The shape of X is (375, 768).

# Q3. Search

In [13]:
scores = X.dot(v)

In [14]:
max(scores)

0.6506573

The highest score in the results is 0.65.

# Q4. Hit-rate for our search engine

In [15]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

In [16]:
search_engine = VectorSearchEngine(documents=ml_docs, embeddings=X)
_ = search_engine.search(v, num_results=5)

In [17]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [18]:
for idx, item in enumerate(ground_truth):
    v = embedding_model.encode(item['question'])
    ground_truth[idx]['v'] = v

In [19]:
ground_truth[0]['v'].dot(embedding_model.encode(ground_truth[0]['question']))

1.0

In [20]:
hit_rate_dict = {}

for idx, item in enumerate(ground_truth):
    search_results = search_engine.search(item['v'], num_results=5)
    hit_rate_dict[ground_truth[idx]['document']] = [d['id'] for d in search_results]

In [21]:
sum([int(k in i) for k, i in hit_rate_dict.items()]) / len(hit_rate_dict)

0.9037433155080213

The hit rate is 90%

# Q5. Indexing with Elasticsearch

In [22]:
from elasticsearch import Elasticsearch

In [23]:
es_client = Elasticsearch('http://localhost:9200')

In [24]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [25]:
from tqdm.auto import tqdm

In [26]:
for idx, doc in enumerate(ml_docs):
    tmp = {
            "text": doc['text'],
            "section": doc['section'],
            "question": doc['question'],
            "course": doc['course'],
            "id": doc['id'],
            "question_text_vector": embeddings[idx] 
    }
    es_client.index(index=index_name, document=tmp)

In [27]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [35]:
highest_matches = elastic_search_knn("question_text_vector", v_q, 'machine-learning-zoomcamp')

In [36]:
highest_matches[0]

{'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp',
 'section': 'General course-related questions',
 'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'id': 'ee58a693'}

The id of the document with the highest score is 'ee58a693'.

# Q6. Hit-rate for Elasticsearch

In [37]:
hit_rate_dict_es = {}

for idx, item in enumerate(ground_truth):
    search_results = elastic_search_knn("question_text_vector", item['v'], 'machine-learning-zoomcamp')
    hit_rate_dict_es[ground_truth[idx]['document']] = [d['id'] for d in search_results]

In [38]:
sum([int(k in i) for k, i in hit_rate_dict_es.items()]) / len(hit_rate_dict_es)

0.9037433155080213

The hit rate stays the same.